In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

Data

In [5]:
df = pd.read_csv("../input/lending-club-20072020q1/Loan_status_2007-2020Q3.gzip",index_col=[0])
df = df[:10000]

In [6]:
df["loan_status"].replace("Fully Paid",1,inplace=True)
df["loan_status"].replace("Current",1,inplace=True)
df["loan_status"].replace("Charged Off",0,inplace=True)

In [7]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

Remove null

In [8]:
null_cols = []
for col in df.columns:
  print(col,df[col].isna().sum())
  if(df[col].isna().sum()>1000):
    null_cols.append(col)
  
df = df.drop(null_cols,axis=1)

df.dropna(inplace=True)

Data continue

In [9]:
y = df_train['loan_status']

# df_train.drop(['loan_status','Unnamed: 0'],axis=1,inplace=True)
df_train.drop(['loan_status','id'],axis=1,inplace=True)

test_index = df_test.index
# df_test.drop('Unnamed: 0',axis=1,inplace=True)
df_test.drop('id',axis=1,inplace=True)

Define Preprocessing steps

In [10]:
numerical_cols= df_train.select_dtypes(exclude='object').columns

catagorical_cols = df_train.select_dtypes(include='object').columns

numerical_transfer = SimpleImputer(strategy='median')

catagorical_transfer = Pipeline(steps=
                                [
                                    ('imputer',SimpleImputer(strategy='most_frequent')),
                                    ('onehot',OneHotEncoder(handle_unknown='ignore'))
                                ])

preprocessor = ColumnTransformer(transformers=
                                 [
                                     ('num',numerical_transfer,numerical_cols),
                                     ('cat',catagorical_transfer,catagorical_cols)
                                 ])

Define the model

In [11]:
from xgboost import XGBRegressor

In [12]:
model = XGBRegressor(n_estimators=1000,random_state=0,learning_rate=0.05)

my_pipeline =Pipeline(steps = 
                     [
                         ('preprocessor',preprocessor),
                         ('model',model)
                     ])

Subbmission

In [13]:
my_pipeline.fit(df_train,y)

pred = my_pipeline.predict(df_test)

output = pd.DataFrame({'id':test_index,
                      'loan_status':pred})

for idx in output.index:
  if(output['loan_status'][idx]>0.5):
    output['loan_status'][idx] = 1
  else:
    output['loan_status'][idx] = 0

output.to_csv('loan-pred.csv')

pd.read_csv('loan-pred.csv', index_col=[0])

Calculate percentage

In [14]:
sum = 0
i=0
for id in  output.id:
    if(output.loan_status[i]==df_test.loan_status[id]):
        sum+=1
    i+=1
    
percentage = sum/len(output)*100
print("Percentage % = "+str(percentage))